In [ ]:
import torch # to create tensors to store the raw data, weights and biases 
import torch.nn as nn # to make the weights and biases learnable (part of the network)
import torch.nn.functional as F # to apply activation functions 
from torch.optim import SGD # stochastic gradient descent 

import lightning as L # to train the model 
from torch.utils.data import DataLoader, TensorDataset # to load the data 

import matplotlib.pyplot as plt # graphs 
import seaborn as sns # graphs
